In [2]:
%matplotlib inline

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
pd. set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 0)

In [4]:
ticker_list = ["aapl", "amzn", "msft"]

ticker = ticker_list[2]

ticker_upper = ticker.upper()

print(ticker_upper)

MSFT


In [5]:
alphavantage_csv_path = f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker_upper}.csv"

In [6]:
df = pd.read_csv(alphavantage_csv_path, low_memory=False)
df.sort_values(by=["date"], ascending=True, inplace=True)

df = df[df["date"]>='2010-01-01']

df["date_time"] = pd.to_datetime(df['date'])

df["day_of_week"] = df["date_time"].dt.dayofweek

# df["adjusted_close_shift"] = df["5. adjusted close"].shift(-1)

# df["return"] = (df["adjusted_close_shift"] - df["5. adjusted close"]) / df["5. adjusted close"] - 1

df["return"] = df["5. adjusted close"].pct_change(1)
df["increase"] = df["return"].apply(lambda x: 1 if x > 0 else 0)

df["log_return"] = np.log(df["return"] + 1)


df.set_index("date", inplace=True)

In [7]:
df.head()



,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2010-01-04,30.62,31.10,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,NaN,0,NaN
2010-01-05,30.85,31.10,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,0.000323,1,0.000323
2010-01-06,30.88,31.08,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,-0.006137,0,-0.006156
2010-01-07,30.63,30.70,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,-0.010335,0,-0.010389
2010-01-08,30.28,30.88,30.24,30.660,23.633805,51197400.0,0.0,1.0,2010-01-08,4,0.006830,1,0.006807


In [8]:
df.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2022-02-28,294.31,299.14,293.000,298.79,298.79,34627457.0,0.0,1.0,2022-02-28,0,0.004978,1,0.004966
2022-03-01,296.40,299.97,292.150,294.95,294.95,31217778.0,0.0,1.0,2022-03-01,1,-0.012852,0,-0.012935
2022-03-02,295.36,301.47,293.698,300.19,300.19,31873007.0,0.0,1.0,2022-03-02,2,0.017766,1,0.017610
2022-03-03,302.89,303.13,294.050,295.92,295.92,27314469.0,0.0,1.0,2022-03-03,3,-0.014224,0,-0.014326
2022-03-04,294.29,295.66,287.165,289.86,289.86,32369655.0,0.0,1.0,2022-03-04,4,-0.020479,0,-0.020691


In [9]:
df["return"] = df["return"].shift(-1)
df["increase"] = df["increase"].shift(-1)


In [10]:
df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2010-01-04,30.62,31.10,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,0.000323,1.0,NaN
2010-01-05,30.85,31.10,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,-0.006137,0.0,0.000323
2010-01-06,30.88,31.08,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,-0.010335,0.0,-0.006156
2010-01-07,30.63,30.70,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,0.006830,1.0,-0.010389
2010-01-08,30.28,30.88,30.24,30.660,23.633805,51197400.0,0.0,1.0,2010-01-08,4,-0.012720,0.0,0.006807


In [11]:
df.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2022-02-28,294.31,299.14,293.000,298.79,298.79,34627457.0,0.0,1.0,2022-02-28,0,-0.012852,0.0,0.004966
2022-03-01,296.40,299.97,292.150,294.95,294.95,31217778.0,0.0,1.0,2022-03-01,1,0.017766,1.0,-0.012935
2022-03-02,295.36,301.47,293.698,300.19,300.19,31873007.0,0.0,1.0,2022-03-02,2,-0.014224,0.0,0.017610
2022-03-03,302.89,303.13,294.050,295.92,295.92,27314469.0,0.0,1.0,2022-03-03,3,-0.020479,0.0,-0.014326
2022-03-04,294.29,295.66,287.165,289.86,289.86,32369655.0,0.0,1.0,2022-03-04,4,NaN,NaN,-0.020691


In [12]:
df.shape[0]

3064

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   1. open               3064 non-null   float64       
 1   2. high               3064 non-null   float64       
 2   3. low                3064 non-null   float64       
 3   4. close              3064 non-null   float64       
 4   5. adjusted close     3064 non-null   float64       
 5   6. volume             3064 non-null   float64       
 6   7. dividend amount    3064 non-null   float64       
 7   8. split coefficient  3064 non-null   float64       
 8   date_time             3064 non-null   datetime64[ns]
 9   day_of_week           3064 non-null   int64         
 10  return                3063 non-null   float64       
 11  increase              3063 non-null   float64       
 12  log_return            3063 non-null   float64       
dtypes: datet

In [14]:
df.head(10)
# df[df.percentage_change == 0]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2010-01-04,30.62,31.10,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,0.000323,1.0,NaN
2010-01-05,30.85,31.10,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,-0.006137,0.0,0.000323
2010-01-06,30.88,31.08,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,-0.010335,0.0,-0.006156
2010-01-07,30.63,30.70,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,0.006830,1.0,-0.010389
2010-01-08,30.28,30.88,30.24,30.660,23.633805,51197400.0,0.0,1.0,2010-01-08,4,-0.012720,0.0,0.006807
2010-01-11,30.71,30.76,30.12,30.270,23.333179,68754700.0,0.0,1.0,2010-01-11,0,-0.006607,0.0,-0.012802
2010-01-12,30.15,30.40,29.91,30.070,23.179012,65912100.0,0.0,1.0,2010-01-12,1,0.009312,1.0,-0.006629
2010-01-13,30.26,30.52,30.01,30.350,23.394846,51863500.0,0.0,1.0,2010-01-13,2,0.020099,1.0,0.009269
2010-01-14,30.31,31.10,30.26,30.960,23.865055,63228100.0,0.0,1.0,2010-01-14,3,-0.003230,0.0,0.019900


In [15]:
df.tail(10)

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2022-02-18,293.05,293.86,286.305,287.93,287.93,34264008.0,0.0,1.0,2022-02-18,4,-0.000729,0.0,-0.009678
2022-02-22,285.00,291.54,284.500,287.72,287.72,41569319.0,0.0,1.0,2022-02-22,1,-0.025893,0.0,-0.000730
2022-02-23,290.18,291.70,280.100,280.27,280.27,37811167.0,0.0,1.0,2022-02-23,2,0.051094,1.0,-0.026234
2022-02-24,272.51,295.16,271.520,294.59,294.59,56989686.0,0.0,1.0,2022-02-24,3,0.009233,1.0,0.049831
2022-02-25,295.14,297.63,291.655,297.31,297.31,32546721.0,0.0,1.0,2022-02-25,4,0.004978,1.0,0.009191
2022-02-28,294.31,299.14,293.000,298.79,298.79,34627457.0,0.0,1.0,2022-02-28,0,-0.012852,0.0,0.004966
2022-03-01,296.40,299.97,292.150,294.95,294.95,31217778.0,0.0,1.0,2022-03-01,1,0.017766,1.0,-0.012935
2022-03-02,295.36,301.47,293.698,300.19,300.19,31873007.0,0.0,1.0,2022-03-02,2,-0.014224,0.0,0.017610
2022-03-03,302.89,303.13,294.050,295.92,295.92,27314469.0,0.0,1.0,2022-03-03,3,-0.020479,0.0,-0.014326


In [16]:
df.increase.value_counts()

1.0    1608
0.0    1455
Name: increase, dtype: int64

In [17]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday
df.day_of_week.value_counts()

1    629
2    628
3    619
4    614
0    574
Name: day_of_week, dtype: int64

In [18]:
loc = df.index.get_loc('2010-01-05')
loc

print(loc)

df.iloc[loc]["date_time"].strftime("%Y-%m-%d")

1


'2010-01-05'

In [19]:
twint_csv_path = f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker}_finbert_twint_20100101_20220304.csv"

In [20]:
twint_csv_path

'/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/MSFT/msft_finbert_twint_20100101_20220304.csv'

In [21]:
df_twint = pd.read_csv(twint_csv_path, low_memory=False, lineterminator='\n')

In [22]:
df_twint.head()

,Headline,Stock,Positive,Negative,Neutral
0,Microsoft: Nobody Will Work With Google Because Of The Nexus One $GOOG by @jwyarow,2010-01-08,0.020996,0.466942,0.512062
1,Here Comes ESPN To Xbox by @ncsaint,2010-01-18,0.026934,0.031920,0.941146
2,Microsoft Readies Secret iPhone Killer $AAPL by @jwyarow,2010-01-19,0.039109,0.021327,0.939564
3,CNBC: Steve Jobs Hates Eric Schmidt $AAPL $GOOG by @jwyarow,2010-01-20,0.026022,0.072853,0.901124
4,Apple In Talks To Replace Google With Bing On The iPhone $AAPL $GOOG by @jwyarow,2010-01-20,0.067153,0.016891,0.915956


In [23]:
df_twint['date'] = pd.to_datetime(df_twint['Stock'])

In [24]:
df_twint["day_of_week"] = df_twint["date"].dt.dayofweek

In [25]:
df_twint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13608 entries, 0 to 13607
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Headline     13598 non-null  object        
 1   Stock        13608 non-null  object        
 2   Positive     13608 non-null  float64       
 3   Negative     13608 non-null  float64       
 4   Neutral      13608 non-null  float64       
 5   date         13608 non-null  datetime64[ns]
 6   day_of_week  13608 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 744.3+ KB


In [26]:
df_twint.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,Microsoft: Nobody Will Work With Google Because Of The Nexus One $GOOG by @jwyarow,2010-01-08,0.020996,0.466942,0.512062,2010-01-08,4


In [27]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

df_twint.day_of_week.value_counts()

1    2510
3    2417
2    2235
0    2221
4    2149
6    1130
5    946 
Name: day_of_week, dtype: int64

In [28]:
df_twint.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,Microsoft: Nobody Will Work With Google Because Of The Nexus One $GOOG by @jwyarow,2010-01-08,0.020996,0.466942,0.512062,2010-01-08,4


In [29]:
df_twint["in_index"] = df_twint["Stock"].apply(lambda x: True if x in df.index else False)

In [30]:
df_twint[df_twint.in_index==False]["day_of_week"].value_counts()

6    1130
5    946 
0    88  
4    37  
1    15  
3    12  
2    7   
Name: day_of_week, dtype: int64

In [31]:
df.index.get_loc('2010-01-04')


0

In [32]:
from datetime import date, datetime, timedelta

In [33]:
test2 =  '2010-01-09'

dtobj1 = datetime.strptime(test2, "%Y-%m-%d")

print(dtobj1)

days = timedelta(2)

print(days)



dtobj2 = (dtobj1 - days).strftime("%Y-%m-%d")
print(dtobj2)


dtobj3 = (dtobj1 + days).strftime("%Y-%m-%d")
print(dtobj3)

2010-01-09 00:00:00
2 days, 0:00:00
2010-01-07
2010-01-11


In [34]:
def update_date(x):

    global ERROR_COUNTER

    if x.in_index == True:
        return x.Stock
    else:
        # try:

            dt_time = datetime.strptime(x.Stock, "%Y-%m-%d")

            # loc = df.index.get_loc(date_part)
            if x.day_of_week == 6: # Sunday to Monday
                days = timedelta(1)
                idx_lookup = (dt_time + days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e1:
                    ERROR_COUNTER += 1
                    print(str(e1))
                    return x.Stock
            if x.day_of_week == 5: # Saturday to Monday
                days = timedelta(2)
                idx_lookup = (dt_time + days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e2:
                    ERROR_COUNTER += 1
                    print(str(e2))
                    return x.Stock
            if x.day_of_week == 4: # Friday to Monday
                days = timedelta(3)
                idx_lookup = (dt_time + days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(3))
                    return x.Stock
            if x.day_of_week == 0: # Monday to Tuesday
                days = timedelta(1)
                idx_lookup = (dt_time + days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(e3))
                    return x.Stock

            else:
                return x.Stock

        # except Exception as e:
        #     ERROR_COUNTER += 1
        #     print(str(e))
        #     print(x.day_of_week, x.Stock, x.in_index)
        #     return x.Stock
    # loc = df.index.get_loc('2010-01-01')
    # loc



In [35]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

ERROR_COUNTER = 0 

df_twint["backfill_date"] = df_twint.apply(update_date, axis=1)


2010-01-19
2010-02-08
2010-02-08
2010-02-15
'2010-02-15'
2010-08-02
2010-09-20
2010-10-04
2010-10-18
2010-10-25
2010-11-15
2010-12-20
2010-12-20
2011-01-10
2011-01-31
2011-02-14
2011-02-14
2011-02-14
2011-02-21
'2011-02-21'
2011-02-21
'2011-02-21'
2011-02-21
'2011-02-21'
2011-03-07
2011-04-04
2011-04-18
2011-04-25
2011-04-25
2011-04-25
2011-04-25
2011-05-02
2011-05-02
2011-05-02
2011-05-23
2011-05-30
'2011-05-30'
2011-06-06
2011-06-13
2011-06-13
2011-06-13
2011-06-13
2011-06-20
2011-06-20
2011-06-20
2011-06-20
2011-07-05
2011-08-01
2011-08-01
2011-08-08
2011-08-08
2011-09-12
2011-09-12
2011-09-12
2011-09-12
2011-09-19
2011-09-19
2011-09-19
2011-09-26
2011-10-03
2011-10-03
2011-10-10
2011-10-24
2011-11-14
2011-11-14
2011-11-21
2011-11-28
2011-11-28
2011-11-28
2011-12-05
2011-12-12
2011-12-12
2011-12-19
2011-12-19
2012-01-03
2012-01-16
'2012-01-16'
2012-01-16
'2012-01-16'
2012-01-16
'2012-01-16'
2012-01-16
'2012-01-16'
2012-01-16
'2012-01-16'
2012-01-30
2012-01-30
2012-02-06
2012-02-21
2

In [36]:
ERROR_COUNTER

133

In [37]:
df_twint["in_index"] = df_twint["backfill_date"].apply(lambda x: True if x in df.index else False)

In [38]:
df_twint[df_twint.in_index==False]["day_of_week"].value_counts()

5    71
6    55
1    15
3    12
2    7 
0    7 
Name: day_of_week, dtype: int64

In [39]:
df_twint[df_twint.in_index==False].shape

(167, 9)

In [40]:
# df_benzinga["text"] = df_benzinga["title"] + " " + df_benzinga["body"]

In [41]:
# df_benzinga['text'] = np.where(df_benzinga["text"], df_benzinga["title"], df_benzinga["text"])

In [42]:
# df_twint = df_twint[["backfill_date", "Positive", "Negative", "Neutral", "Retweets"]]
df_twint = df_twint[["backfill_date", "Positive", "Negative", "Neutral"]]

In [43]:
df_twint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13608 entries, 0 to 13607
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   backfill_date  13608 non-null  object 
 1   Positive       13608 non-null  float64
 2   Negative       13608 non-null  float64
 3   Neutral        13608 non-null  float64
dtypes: float64(3), object(1)
memory usage: 425.4+ KB


In [44]:
df_twint.head()

,backfill_date,Positive,Negative,Neutral
0,2010-01-08,0.020996,0.466942,0.512062
1,2010-01-19,0.026934,0.031920,0.941146
2,2010-01-19,0.039109,0.021327,0.939564
3,2010-01-20,0.026022,0.072853,0.901124
4,2010-01-20,0.067153,0.016891,0.915956


In [45]:
df_twint.rename(columns={"backfill_date": "date"}, inplace=True)

In [46]:
def clean_tweet(df):
    # replace URLs with a whitespace
    df['text'] = df['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    
    return df

In [47]:
# df_twint = clean_tweet(df_twint)

In [48]:
df_twint.set_index("date", inplace=True)

In [49]:
df_twint.head()

,Positive,Negative,Neutral
date,,,
2010-01-08,0.020996,0.466942,0.512062
2010-01-19,0.026934,0.031920,0.941146
2010-01-19,0.039109,0.021327,0.939564
2010-01-20,0.026022,0.072853,0.901124
2010-01-20,0.067153,0.016891,0.915956


In [50]:
df_twint_duplicated_index = df_twint[df_twint.index.duplicated(keep=False)]

In [51]:
df_twint_duplicated_index.shape[0]

13055

In [52]:
df_twint_nonduplicated_index = df_twint[~df_twint.index.duplicated(keep=False)]

In [53]:
df_twint_nonduplicated_index.shape[0]

553

In [54]:
# df_twint.groupby('date')['negative'].mean()

df_twint_groupby = df_twint.groupby("date").agg(
     negative = ("Negative", "mean"),
     nuetral = ("Neutral", "mean"),
     positive = ("Positive", "mean"),
     # retweets = ("Retweets", "sum"),
          # compound = ("compound", "mean"),
          # nlikes = ("nlikes", "sum"),
          # nretweets = ("nretweets", "sum"),
     )


In [55]:
df_twint_groupby

,negative,nuetral,positive
date,,,
2010-01-08,0.466942,0.512062,0.020996
2010-01-19,0.026624,0.940355,0.033021
2010-01-20,0.044872,0.908540,0.046587
2010-02-03,0.041638,0.927986,0.030376
2010-02-08,0.272172,0.690500,0.037328
...,...,...,...
2022-03-01,0.044504,0.904235,0.051261
2022-03-02,0.094913,0.678821,0.226266
2022-03-03,0.031622,0.765418,0.202961


In [56]:
df_merge = pd.merge(df, df_twint_groupby, how="left", left_index=True, right_index=True)

In [57]:
df_merge.shape

(3064, 16)

In [58]:
df_merge.day_of_week.value_counts()

1    629
2    628
3    619
4    614
0    574
Name: day_of_week, dtype: int64

In [59]:
# df.index
# df_text.index
df_merge[df_merge.negative.isnull()]
# df_merge.shape

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,30.620,31.1000,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,0.000323,1.0,NaN,NaN,NaN,NaN
2010-01-05,30.850,31.1000,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,-0.006137,0.0,0.000323,NaN,NaN,NaN
2010-01-06,30.880,31.0800,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,-0.010335,0.0,-0.006156,NaN,NaN,NaN
2010-01-07,30.630,30.7000,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,0.006830,1.0,-0.010389,NaN,NaN,NaN
2010-01-11,30.710,30.7600,30.12,30.270,23.333179,68754700.0,0.0,1.0,2010-01-11,0,-0.006607,0.0,-0.012802,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30,285.710,287.8300,281.62,281.920,280.824178,32343635.0,0.0,1.0,2021-09-30,3,0.025468,1.0,-0.007351,NaN,NaN,NaN
2021-10-08,296.219,296.6409,293.76,294.850,293.703919,17685700.0,0.0,1.0,2021-10-08,4,-0.002103,0.0,0.000000,NaN,NaN,NaN
2021-11-11,331.250,333.7746,330.51,332.430,331.137846,16849844.0,0.0,1.0,2021-11-11,3,0.012905,1.0,0.004915,NaN,NaN,NaN


In [60]:
df.shape

(3064, 13)

In [61]:
df_twint_groupby.shape

(2548, 3)

In [62]:
df_merge.fillna(method="ffill", limit=1, inplace=True)

In [63]:
df_merge[df_merge.negative.isnull()]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,30.62,31.10,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,0.000323,1.0,NaN,NaN,NaN,NaN
2010-01-05,30.85,31.10,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,-0.006137,0.0,0.000323,NaN,NaN,NaN
2010-01-06,30.88,31.08,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,-0.010335,0.0,-0.006156,NaN,NaN,NaN
2010-01-07,30.63,30.70,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,0.006830,1.0,-0.010389,NaN,NaN,NaN
2010-01-12,30.15,30.40,29.91,30.070,23.179012,65912100.0,0.0,1.0,2010-01-12,1,0.009312,1.0,-0.006629,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-08,109.33,109.75,108.76,109.490,104.779461,15487502.0,0.0,1.0,2018-08-08,2,0.001644,1.0,0.005587,NaN,NaN,NaN
2018-08-17,107.36,107.90,106.69,107.580,103.353266,18061512.0,0.0,1.0,2018-08-17,4,-0.006600,0.0,-0.000558,NaN,NaN,NaN
2018-09-12,111.43,111.85,110.51,111.710,107.321001,18891064.0,0.0,1.0,2018-09-12,2,0.010742,1.0,0.004216,NaN,NaN,NaN


In [64]:
df_merge.fillna(0.0, inplace=True)

In [65]:
# df_merge = df_merge.sample(frac=1).reset_index(drop=True)
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,30.62,31.10,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,0.000323,1.0,0.000000,0.000000,0.000000,0.000000
2010-01-05,30.85,31.10,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,-0.006137,0.0,0.000323,0.000000,0.000000,0.000000
2010-01-06,30.88,31.08,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,-0.010335,0.0,-0.006156,0.000000,0.000000,0.000000
2010-01-07,30.63,30.70,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,0.006830,1.0,-0.010389,0.000000,0.000000,0.000000
2010-01-08,30.28,30.88,30.24,30.660,23.633805,51197400.0,0.0,1.0,2010-01-08,4,-0.012720,0.0,0.006807,0.466942,0.512062,0.020996


In [66]:
# df_merge["score"] = (df_merge["positive"] - df_merge["negative"])

In [67]:
# X = df_merge[["negative", "nuetral", "positive"]]
# X = df_merge[["5. adjusted close", "2. high", "3. low", "6. volume", "negative", "nuetral", "positive"]]
X = df_merge[["log_return", "positive"]]
y = df_merge["increase"]

In [68]:
y.head()

date
2010-01-04    1.0
2010-01-05    0.0
2010-01-06    0.0
2010-01-07    1.0
2010-01-08    0.0
Name: increase, dtype: float64

In [69]:
SPLIT = int(0.9 * len(df_merge))

X_train = X[:SPLIT]
X_test = X[SPLIT:]

y_train = y[:SPLIT]
y_test = y[SPLIT:]



In [70]:
X_train

,log_return,positive
date,,
2010-01-04,0.000000,0.000000
2010-01-05,0.000323,0.000000
2010-01-06,-0.006156,0.000000
2010-01-07,-0.010389,0.000000
2010-01-08,0.006807,0.020996
...,...,...
2020-12-08,0.007994,0.167565
2020-12-09,-0.019682,0.382371
2020-12-10,-0.006062,0.057050


In [71]:
from sklearn.ensemble import RandomForestClassifier

In [72]:
rfc = RandomForestClassifier(n_estimators=200, criterion="entropy")
rfc.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [73]:
preds = rfc.predict(X_test)

In [74]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [75]:
matrix = confusion_matrix(y_test, preds)

In [76]:
matrix

array([[78, 69],
       [73, 87]])

In [77]:
score = accuracy_score(y_test, preds)

In [78]:
score

0.5374592833876222

In [79]:
y_test.value_counts()

1.0    160
0.0    147
Name: increase, dtype: int64

In [80]:
y_test.value_counts(normalize=True)

1.0    0.521173
0.0    0.478827
Name: increase, dtype: float64

In [81]:
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

         0.0       0.52      0.53      0.52       147
         1.0       0.56      0.54      0.55       160

    accuracy                           0.54       307
   macro avg       0.54      0.54      0.54       307
weighted avg       0.54      0.54      0.54       307



In [82]:
# corr_matrix = df_merge[["negative", "nuetral", "positive", "retweets", "increase"]].corr()
corr_matrix = df_merge[["log_return", "positive", "increase"]].corr()
print (corr_matrix)

            log_return  positive  increase
log_return  1.000000    0.079869 -0.056121
positive    0.079869    1.000000  0.014580
increase   -0.056121    0.014580  1.000000


In [83]:
from statsmodels.tsa.stattools import grangercausalitytests

In [84]:
#perform Granger-Causality test
# grangercausalitytests(df_merge[["positive", "increase"]], maxlag=[63])

In [85]:
df_merge.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2022-02-28,294.31,299.14,293.000,298.79,298.79,34627457.0,0.0,1.0,2022-02-28,0,-0.012852,0.0,0.004966,0.068048,0.862345,0.069607
2022-03-01,296.40,299.97,292.150,294.95,294.95,31217778.0,0.0,1.0,2022-03-01,1,0.017766,1.0,-0.012935,0.044504,0.904235,0.051261
2022-03-02,295.36,301.47,293.698,300.19,300.19,31873007.0,0.0,1.0,2022-03-02,2,-0.014224,0.0,0.017610,0.094913,0.678821,0.226266
2022-03-03,302.89,303.13,294.050,295.92,295.92,27314469.0,0.0,1.0,2022-03-03,3,-0.020479,0.0,-0.014326,0.031622,0.765418,0.202961
2022-03-04,294.29,295.66,287.165,289.86,289.86,32369655.0,0.0,1.0,2022-03-04,4,-0.020479,0.0,-0.020691,0.327597,0.614870,0.057533


In [86]:
df_merge["return"].mean(), df_merge["return"].std()

(0.0009343558602451958, 0.01585859273755356)

In [87]:
df_merge["log_return"].mean(), df_merge["log_return"].std()

(0.000815047597794545, 0.015857222351919727)

In [88]:
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,30.62,31.10,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,0.000323,1.0,0.000000,0.000000,0.000000,0.000000
2010-01-05,30.85,31.10,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,-0.006137,0.0,0.000323,0.000000,0.000000,0.000000
2010-01-06,30.88,31.08,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,-0.010335,0.0,-0.006156,0.000000,0.000000,0.000000
2010-01-07,30.63,30.70,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,0.006830,1.0,-0.010389,0.000000,0.000000,0.000000
2010-01-08,30.28,30.88,30.24,30.660,23.633805,51197400.0,0.0,1.0,2010-01-08,4,-0.012720,0.0,0.006807,0.466942,0.512062,0.020996


In [89]:
df_merge.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2022-02-28,294.31,299.14,293.000,298.79,298.79,34627457.0,0.0,1.0,2022-02-28,0,-0.012852,0.0,0.004966,0.068048,0.862345,0.069607
2022-03-01,296.40,299.97,292.150,294.95,294.95,31217778.0,0.0,1.0,2022-03-01,1,0.017766,1.0,-0.012935,0.044504,0.904235,0.051261
2022-03-02,295.36,301.47,293.698,300.19,300.19,31873007.0,0.0,1.0,2022-03-02,2,-0.014224,0.0,0.017610,0.094913,0.678821,0.226266
2022-03-03,302.89,303.13,294.050,295.92,295.92,27314469.0,0.0,1.0,2022-03-03,3,-0.020479,0.0,-0.014326,0.031622,0.765418,0.202961
2022-03-04,294.29,295.66,287.165,289.86,289.86,32369655.0,0.0,1.0,2022-03-04,4,-0.020479,0.0,-0.020691,0.327597,0.614870,0.057533


In [90]:
df_merge["increase"] = df_merge["increase"].shift(1)
df_merge["return"] = df_merge["return"].shift(1)

In [91]:
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,30.62,31.10,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,NaN,NaN,0.000000,0.000000,0.000000,0.000000
2010-01-05,30.85,31.10,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,0.000323,1.0,0.000323,0.000000,0.000000,0.000000
2010-01-06,30.88,31.08,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,-0.006137,0.0,-0.006156,0.000000,0.000000,0.000000
2010-01-07,30.63,30.70,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,-0.010335,0.0,-0.010389,0.000000,0.000000,0.000000
2010-01-08,30.28,30.88,30.24,30.660,23.633805,51197400.0,0.0,1.0,2010-01-08,4,0.006830,1.0,0.006807,0.466942,0.512062,0.020996


In [92]:
df_merge.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2022-02-28,294.31,299.14,293.000,298.79,298.79,34627457.0,0.0,1.0,2022-02-28,0,0.004978,1.0,0.004966,0.068048,0.862345,0.069607
2022-03-01,296.40,299.97,292.150,294.95,294.95,31217778.0,0.0,1.0,2022-03-01,1,-0.012852,0.0,-0.012935,0.044504,0.904235,0.051261
2022-03-02,295.36,301.47,293.698,300.19,300.19,31873007.0,0.0,1.0,2022-03-02,2,0.017766,1.0,0.017610,0.094913,0.678821,0.226266
2022-03-03,302.89,303.13,294.050,295.92,295.92,27314469.0,0.0,1.0,2022-03-03,3,-0.014224,0.0,-0.014326,0.031622,0.765418,0.202961
2022-03-04,294.29,295.66,287.165,289.86,289.86,32369655.0,0.0,1.0,2022-03-04,4,-0.020479,0.0,-0.020691,0.327597,0.614870,0.057533


In [93]:
df_merge.fillna(0.0, inplace=True)

In [94]:
# df_merge.to_csv(f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker_upper}_WITH_TWINT_FINBERT_SA.csv")